In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
import csv
import os
import pandas as pd

In [2]:
# Set up the web driver and the url
starbucks_url = "https://www.starbucks.com/store-locator?"
driver = webdriver.Chrome()
driver.get(starbucks_url);
sleep(20)

In [3]:
# Access url
input_formula = driver.find_element("name", "place")
location = input("Enter Your Location: ")
input_formula.send_keys(location)
input_formula.send_keys(Keys.ENTER)

Enter Your Location: Dallas, Tx


In [4]:
# Scrape for each restuarant's information and store that in an object
restaurant_links = driver.find_elements(By.XPATH, '//a[@href="/store-locator/store/"]')
results_length = len(restaurant_links)
results = []

for i in range(results_length):
    driver.execute_script("arguments[0].click();", restaurant_links[i])
    # Restaurant name
    try:
        get_restaurant_name = driver.find_elements(By.CLASS_NAME, "sb-heading.text-lg.text-bold.pr7.mb1")
        name = get_restaurant_name[0].get_attribute("innerHTML")
    except Exception as e:
        print(e)

     # Address
    try:
        get_address_name = driver.find_element(By.CLASS_NAME, "pb5.grid .gridItem.size6of12")
        address_lines = get_address_name.find_elements(By.TAG_NAME, "span")
        address = [line.text for line in address_lines]
        address = ", ".join(word for word in address)
    except Exception as e:
        print(e)

    # Phone number
    try:
        get_phone_number = driver.find_element(By.CLASS_NAME, "sb-button.sb-button--default.mb3.lg-mxn3")
        phone_number_href = get_phone_number.get_attribute("href")
        phone_number = phone_number_href.split("tel:")[1]
    except Exception as e:
        print(e)

    # Store schedule
    try:
        get_store_days = driver.find_elements(By.CLASS_NAME, "scheduleDay___mA9Wh")
        get_store_hours = driver.find_elements(By.CLASS_NAME, "scheduleHours___ddcX1")

        store_schedule = []
        if len(get_store_days) == len(get_store_hours):
            for j in range(len(get_store_days)):
                store_day = get_store_days[j].get_attribute("innerHTML")
                store_hours = get_store_hours[j].get_attribute("innerHTML")
                store_schedule.append({store_day: store_hours})
        else:
            raise MismatchDaysAndHoursError("Mismatch between number of days and hours in store schedule.") 

    except Exception as e:
        print(e)

    # Amenities and Pickup Options
    try:
        get_amenities = driver.find_elements(By.CLASS_NAME, "featureListItem___Sd3tQ")
        amenities = []

        for amenity in get_amenities:
            amenity_flex_grow = amenity.find_element(By.CLASS_NAME, "flex-grow")
            amenity = amenity_flex_grow.get_attribute("innerHTML")
            amenities.append(amenity)


    except Exception as e:
        print(e)          

    results.append({"name": name,
            "address": address,
            "phone_number": phone_number,
            "store_schedule": store_schedule,
            "amenities": amenities,
           })
  
driver.close()
    
        
        


list index out of range
Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Sessio

Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Session info: chrome=128.0.6613.114)

list index out of range
Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such eleme

Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Session info: chrome=128.0.6613.114)

list index out of range
Message: no such element: Unable to locate element: {"method":"css selector","selector":".pb5.grid .gridItem.size6of12"}
  (Session info: chrome=128.0.6613.114)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".sb-button.sb-button--default.mb3.lg-mxn3"}
  (Session info: chrome=128.0.6613.114)



In [5]:
# Flatten the the name, address, and phone_number
restaurants = []
for result in results:
    # Flatten the the name, address, and phone_number
    restaurant = {
        "name": result["name"],
        "address": result["address"],
        "phone_number": result["phone_number"],
    }
    
    # Flatten the the store_schedule
    for day_schedule in result["store_schedule"]:
        for day, schedule in day_schedule.items():
            restaurant[day] = schedule
    
    # Flatten the amenities
    restaurant["amenities"] = ", ".join(result["amenities"]) 
    
    restaurants.append(restaurant)




In [6]:
# Writing the data to starbucks.csv
if restaurants:
    fieldnames = list(restaurants[0].keys())

file_exists = os.path.isfile("starbucks.csv")
with open("starbucks.csv", mode="a", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()
    writer.writerows(restaurants)

In [7]:
# Write to rastaurants.csv
for restaurant in restaurants:
    restaurant["restaurant_type"] = "Starbucks"
fieldnames = ["name", "address", "restaurant_type"]
filtered_restaurants = [
    {key: restaurant[key] for key in fieldnames if key in restaurant}
    for restaurant in restaurants
]

file_exists = os.path.isfile("restaurants.csv")
with open("restaurants.csv", mode="a", newline="") as output_file:
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()
    writer.writerows(filtered_restaurants)
